In [1]:
import pandas as pd

Aiming to combine 2018 and 2020 CCG -> STP -> region lookup

To account for CCG changes

In [2]:
ccg19 = pd.read_csv('../data/ccg_2019.csv')
ccg19 = ccg19[['CCG19CDH', 'CCG19NM', 'STP19CD', 'STP19NM']] \
        .rename(columns = {'CCG19CDH': 'ccg_code', 'CCG19NM': 'ccg_name', 'STP19CD': 'stp_code', 'STP19NM':'stp_name'})
ccg19['file'] = '2019 CCGs'

ccg20 = pd.read_csv('../data/ccg_2020.csv')
ccg20 = ccg20[['CCG20CDH', 'CCG20NM', 'STP20CD', 'STP20NM']] \
        .rename(columns = {'CCG20CDH': 'ccg_code', 'CCG20NM': 'ccg_name', 'STP20CD': 'stp_code', 'STP20NM':'stp_name'})
ccg20['file'] = '2020 CCGs'

ccg21 = pd.read_csv('../data/ccg_2021.csv')
ccg21 = ccg21[['CCG21CDH', 'CCG21NM', 'STP21CD', 'STP21NM']] \
        .rename(columns = {'CCG21CDH': 'ccg_code', 'CCG21NM': 'ccg_name', 'STP21CD': 'stp_code', 'STP21NM':'stp_name'})
ccg21['file'] = '2021 CCGs'

stp = pd.read_csv('../data/stp_region_21.csv')


In [3]:
ccg_concat = pd.concat([ccg19, ccg20, ccg21])

In [4]:
ccg_concat.sample()

,ccg_code,ccg_name,stp_code,stp_name,file
123,08Q,NHS Southwark CCG,E54000030,Our Healthier South East London,2019 CCGs


In [5]:
stp.sample()

,FID,STP20CD,STP20CDH,STP20NM,NHSER20CD,NHSER20CDH,NHSER20NM
9,10,E54000052,QXU,Surrey Heartlands Health and Care Partnership,E40000005,Y59,South East


In [6]:
ccg_concat.shape

(432, 5)

In [7]:
ccg_concat.stp_code = ccg_concat.stp_code.replace('E54000005', 'E54000054')
ccg_concat.stp_code = ccg_concat.stp_code.replace('E54000006', 'E54000051')
ccg_concat.stp_code = ccg_concat.stp_code.replace('E54000033', 'E54000053')
ccg_concat.stp_code = ccg_concat.stp_code.replace('E54000035', 'E54000052')
ccg_concat.stp_code = ccg_concat.stp_code.replace('E54000049', 'E54000050')

ccg_concat = ccg_concat.drop_duplicates(subset=['ccg_code', 'ccg_name', 'stp_code', 'stp_name'])

In [8]:
end_lookup = ccg_concat.merge(stp, left_on='stp_code', right_on='STP20CD', how='left')

end_lookup.stp_name = end_lookup.stp_name.replace('Cornwall and the Isles of Scilly Health and Social Care Partnership', 'Cornwall and the Isles of Scilly')
end_lookup.stp_name = end_lookup.stp_name.replace('Frimley Health and Care ICS', 'Frimley Health')
end_lookup.stp_name = end_lookup.stp_name.replace('Surrey Heartlands Health and Care Partnership', 'Surrey Heartlands')
end_lookup.stp_name = end_lookup.stp_name.replace('Sussex and East Surrey Health and Care Partnership', 'Sussex Health and Care Partnership')
end_lookup.stp_name = end_lookup.stp_name.replace('Sussex and East Surrey', 'Sussex Health and Care Partnership')

In [9]:
end_lookup.to_csv('../outputs/combined_lookup.csv')

## Now add to db in heroku

In [15]:
from sqlalchemy import Table, Column, Integer, String,DateTime,DateTime, MetaData,create_engine, Float
from sqlalchemy.ext.declarative import declarative_base
from dotmap import DotMap
from pathlib import Path
from dotenv import load_dotenv
import os

In [16]:
env_path = Path('/Users/jackpickard/Documents/Analytics/location-lookup-api/')
load_dotenv(dotenv_path=env_path)

True

In [22]:
db_url = os.environ.get('PRODUCTION_DB_URL')

In [23]:
engine = create_engine(db_url)

In [24]:
end_lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ccg_code    226 non-null    object
 1   ccg_name    226 non-null    object
 2   stp_code    226 non-null    object
 3   stp_name    226 non-null    object
 4   file        226 non-null    object
 5   FID         226 non-null    int64 
 6   STP20CD     226 non-null    object
 7   STP20CDH    226 non-null    object
 8   STP20NM     226 non-null    object
 9   NHSER20CD   226 non-null    object
 10  NHSER20CDH  226 non-null    object
 11  NHSER20NM   226 non-null    object
dtypes: int64(1), object(11)
memory usage: 23.0+ KB


In [25]:
lookup_cols = dict(
    ccg_code = 'ccg_code',
    ccg_name = 'ccg_name',
    stp_code = 'stp_code',
    stp_name = 'stp_name',
    file = 'file',
    FID = 'FID',
    STP20CD = 'STP20CD',
    STP20CDH = 'STP20CDH',
    STP20NM = 'STP20NM',
    NHSER20CD = 'NHSER20CD',
    NHSER20CDH = 'NHSER20CDH',
    NHSER20NM = 'NHSER20NM'
)

cd = DotMap(lookup_cols)

dtypes={cd.ccg_code : String(50),
        cd.ccg_name: String(500),
        cd.stp_code: String(50),
        cd.stp_name: String(500),
        cd.file: String(100),
        cd.FID: String(100),
        cd.STP20CD: String(100),
        cd.STP20CDH: String(100),
        cd.STP20NM: String(100),
        cd.NHSER20CD: String(100),
        cd.NHSER20CDH: String(100),
        cd.NHSER20NM: String(100),
        }

In [26]:
end_lookup.to_sql("lookup",
               engine,
               if_exists='replace',
               index=True,
            index_label = 'id',
               chunksize=10000,
               method='multi',
               dtype=dtypes)